In [ ]:
# Cell 1: Imports
import sys
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
sys.path.append(os.path.abspath(os.path.join('..')))
from src.classification_library import load_data, clean_data
from src.timeseries_library import check_stationarity, fit_arima

# Cell 2: Parameters
# tags=["parameters"]
RAW_ZIP_PATH = '../data/raw/PRSA2017_Data_20130301-20170228.zip'
STATION = 'Aotizhongxin'
VALUE_COL = 'PM2.5'
CUTOFF = '2017-01-01'
P_MAX = 3
Q_MAX = 3
D_MAX = 2
IC = 'aic'
ARTIFACTS_PREFIX = 'arima_pm25'

# Cell 3: Main Execution
print("--- BẮT ĐẦU BƯỚC 3: ARIMA FORECASTING ---")

# 1. Load Data
df = clean_data(load_data(RAW_ZIP_PATH))

# 2. Chuẩn bị chuỗi thời gian (Resample theo giờ để đảm bảo liên tục)
print(f"Đang xử lý trạm: {STATION}")
series = df[df['station'] == STATION][VALUE_COL]
series = series.asfreq('h').fillna(method='ffill') # Điền dữ liệu thiếu

# 3. Chia Train/Test
train = series[series.index < CUTOFF]
test = series[series.index >= CUTOFF]
print(f"Train samples: {len(train)}, Test samples: {len(test)}")

# 4. Kiểm tra tính dừng (Stationarity)
stat_result = check_stationarity(train)
print("Kết quả kiểm định ADF:", stat_result)

# 5. Fit ARIMA (Ví dụ cố định order=(2,1,2) cho đơn giản, thực tế có thể grid search)
# Lưu ý: Chạy ARIMA trên tập dữ liệu lớn tốn nhiều thời gian
print("Đang huấn luyện ARIMA(2,1,2)... (Có thể mất vài phút)")
model_fit = fit_arima(train, order=(2,1,2)) 
print(model_fit.summary())

# 6. Forecast
print("Đang dự báo...")
forecast_res = model_fit.forecast(steps=len(test))
print("Dự báo hoàn tất.")